In [1]:
%pylab inline
import scipy.stats
from collections import defaultdict  # default dictionary 
import copy
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
class Node:
    def __init__(self,purity,klasslabel='',score=0,split=[],fidx=-1):
        self.lchild=None       
        self.rchild=None
        self.klasslabel=klasslabel        
        self.split=split
        self.score=score
        self.fidx=fidx
        self.purity=purity    
        
    def set_childs(self,lchild,rchild):
        self.lchild=lchild
        self.rchild=rchild

        
    def isleaf(self):
        if(self.lchild==None and self.lchild==None):
            return True
        else:
            return False
        # Your Code Here
        

    def isless_than_eq(self, X):
        if (X[self.fidx] < self.split):
            return True
        return False
        # Your Code Here

        
    def get_str(self):        
        if self.isleaf():
            return 'C(class={},Purity={})'.format(self.klasslabel,self.purity)
        else:
            return 'I(Fidx={},Score={},Split={})'.format(self.fidx,self.score,self.split)

In [3]:
def calculate_entropy(P1,P2,P3):
    if (P1 == 0):
        P1 += 0.0000001
    if (P2 == 0):
        P2 += 0.0000001
    if (P3 == 0):
        P3 += 0.0000001
    return -((P1*np.log2(P1))+(P2*np.log2(P2))+(P3*np.log2(P3)))

In [4]:
## Your code goes here...7852
class DecisionTree:
    ''' Implements the Decision Tree For Classification... '''
    def __init__(self, purityp, exthreshold,maxdepth=10,tree=None):        
        self.purity=purityp
        self.exthreshold=exthreshold
        self.maxdepth=maxdepth
        self.tree=tree
        
    def train(self, X, Y):
        ''' Train Decision Tree using the given 
            X [m x d] data matrix and Y labels matrix
            
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns:
            -----------
            Nothing
            '''
        nexamples,nfeatures=X.shape
        ## now go and train a model for each class...
        # YOUR CODE HERE
        self.tree = self.build_tree(X,Y,self.maxdepth)
        #self.__str__()
        
        
        
    def build_tree(self, X, Y, depth):
        """ 
            Function is used to recursively build the decision Tree 
          
            Input
            -----
            X: [m x d] a data matrix of m d-dimensional examples.
            Y: [m x 1] a label vector.
            
            Returns
            -------
            root node of the built tree...
        """
        nexamples, nfeatures=X.shape
      
        klasses,counts=np.unique(Y,return_counts=True);
        # YOUR CODE HERE                
        class_purity = counts[np.argmax(counts)]/np.sum(counts)
        if(depth==0 or self.purity<=class_purity):
            return Node(class_purity, klasses[np.argmax(counts)])
        best_split = 0.0
        best_score = 0.0
        feature = -1
        leftIdx = None
        rightIdx = None
        
        # Getting best feature and score
        for i in range (0, X.shape[1]):
            
            split,mingain,Xlidx,Xridx=self.evaluate_numerical_attribute(X[:,i],Y)
            
            if(mingain > best_score):
                best_score = mingain
                best_split = split
                feature = i
                leftIdx = Xlidx
                rightIdx = Xridx
       
        # Recursively calling build_tree on nodes
        n = Node(purity=class_purity,klasslabel='', score=best_score, split=best_split, fidx=feature)
        
        n.lchild = self.build_tree(X[leftIdx], Y[leftIdx], depth-1)
        n.rchild = self.build_tree(X[rightIdx], Y[rightIdx], depth-1)
        return n
        
    def test(self, X):
        
        ''' Test the trained classifiers on the given set of examples 
        
                   
            Input:
            ------
            X: [m x d] a data matrix of m d-dimensional test examples.
           
            Returns:
            -----------
                pclass: the predicted class for each example, i.e. to which it belongs
        '''
        
        nexamples, nfeatures=X.shape
        pclasses=self.predict(X)
        
        # your code go here...
        
    
    def evaluate_numerical_attribute(self,feat, Y):
        '''
            Evaluates the numerical attribute for all possible split points for
            possible feature selection
            
            Input:
            ---------
            feat: a contiuous feature
            Y: labels
            
            Returns:
            ----------
            v: splitting threshold
            score: splitting score
            Xlidx: Index of examples belonging to left child node
            Xridx: Index of examples belonging to right child node
            
        '''
        
        # A big source of Bugs will be sorting the same array and expecting it to behave original,
        # use separate variables to store the sorted array and its corresponding classes labels...
        
        classes=np.unique(Y)
        nclasses=len(classes)
        sidx=np.argsort(feat)
        f=feat[sidx] # sorted features
        sY=Y[sidx] # sorted features class labels...
        
        # YOUR CODE HERE
        
        #for i in get_Spliting_values_from_column(feat):
        classes,counts = np.unique(Y,return_counts = True)
        Target_entropy = 0.0
        split = 0.0
        score = 0.0
        
        for count in counts:
            Target_entropy +=  -(((count/np.sum(counts))*(math.log2(count/np.sum(counts)))))
            
        SplitingArray = []
        for i in range(len(np.unique(feat))):
            SplitingArray.append((np.unique(feat)[i-1]+np.unique(feat)[i])/2)
        prev_entropy = 999
        for x in SplitingArray:
            P1=P2=P3=P4=P5=P6=0
            entropy=0.0
            for i in range(len(feat)):
                if(x<feat[i] and Y[i] == classes[0]):
                    P1 += 1
                elif(x < feat[i] and Y[i] == classes[1]):
                    P2 += 1
                elif(x > feat[i] and Y[i] == classes[0]):
                    P4 += 1
                elif(x > feat[i] and Y[i] == classes[1]):
                    P5 += 1
                elif(x < feat[i]):
                    P3 += 1
                elif(x > feat[i]):
                    P6 += 1            
            if((P1+P2+P3)==0 or (P4+P5+P6) ==0):
                continue
            entropy = ((((P1+P2+P3)/np.sum(counts))*calculate_entropy(P1/(P1+P2+P3),P2/(P1+P2+P3),P3/(P1+P2+P3)))+(((P4+P5+P6)/np.sum(counts))*calculate_entropy(P4/(P4+P5+P6),P5/(P4+P5+P6),P6/(P4+P5+P6))))
            #print(P1,P2,P3,P4,P5,P6,entropy,x)
            if(entropy <= prev_entropy):
                prev_entropy = entropy
                score =(Target_entropy - entropy)
                split = x
        leftChildInd = np.where(feat <  split)[0]
        RightChildInd = np.where(feat > split)[0]
        return split, score, leftChildInd, RightChildInd
    
    def predict(self, X):
        
        """
        Test the trained classifiers on the given example X
        
                   
            Input:
            ------
            X: [1 x d] a d-dimensional test example.
           
            Returns:
            -----------
                pclass: the predicted class for the given example, i.e. to which it belongs
        """
       #  # YOUR CODE HERE
        pclass = []
        for i in range (0, X.shape[0]):
            temp = self._predict(self.tree, X[i,:])
            pclass.append(temp)
        return pclass
    
    def _predict(self,node, X):
        if (node.isleaf() == True):
            temp = node.klasslabel
            return temp
        else:
            if (node.isless_than_eq(X) == True):
                return self._predict(node.lchild, X)
            else:
                return self._predict(node.rchild, X)
        # YOUR CODE HERE
      

        

    def __str__(self):
        
        return self.__print(self.tree)        
        
     
    def find_depth(self):
        
        return self._find_depth(self.tree)
    
    
    def _find_depth(self,node):
        if not node:
            return
        if node.isleaf():
            return 1
        else:
            return max(self._find_depth(node.lchild),self._find_depth(node.rchild))+1
        
    def __print(self,node,depth=0):
        
        ret = ""

        # Print right branch
        if node.rchild:
            ret += self.__print(node.rchild,depth+1)

        # Print own value
        
        ret += "\n" + ("    "*depth) + node.get_str()

        # Print left branch
        if node.lchild:
            ret += self.__print(node.lchild,depth+1)
        
        return ret

In [5]:
import pandas as pd
import tools as t # set of tools for plotting, data splitting, etc..

In [6]:
#load the data set
data=pd.read_csv('./iris.data')
data.columns=['SepalLength','SepalWidth','PetalLength','PetalWidth','Class']
print (data.describe())

       SepalLength  SepalWidth  PetalLength  PetalWidth
count   149.000000  149.000000   149.000000  149.000000
mean      5.848322    3.051007     3.774497    1.205369
std       0.828594    0.433499     1.759651    0.761292
min       4.300000    2.000000     1.000000    0.100000
25%       5.100000    2.800000     1.600000    0.300000
50%       5.800000    3.000000     4.400000    1.300000
75%       6.400000    3.300000     5.100000    1.800000
max       7.900000    4.400000     6.900000    2.500000


In [7]:
# Get your data in matrix (X ,Y)
X = np.array(data.loc[:, 'SepalLength':'PetalWidth'])
Y = np.array(data.loc[:,'Class'])
print (" Data Set Dimensions=", X.shape, " True Class labels dimensions", Y.shape)

 Data Set Dimensions= (149, 4)  True Class labels dimensions (149,)


In [8]:
dt=DecisionTree(0.95,5,2)
feat=[0,1]
dt.classes=np.unique(Y)
dt.nclasses=len(np.unique(Y))
split,mingain,Xlidx,Xridx=dt.evaluate_numerical_attribute(X[:,3],Y)
print(split,mingain)

0.8 0.9137533408759091


In [9]:
print (len(Y))
dt=DecisionTree(0.95,5)
dt.train(X, Y)
g,s,xl,xr=dt.evaluate_numerical_attribute(X[:,2],Y)
print(g, s, xl, xr)

149
2.45 0.9137533408759091 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48] [ 49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66
  67  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101 102
 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120
 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138
 139 140 141 142 143 144 145 146 147 148]


In [11]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

print (" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape)
print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytrain.shape)   

 Training Data Set Dimensions= (104, 4) Training True Class labels dimensions (104,)
 Test Data Set Dimensions= (45, 4) Test True Class labels dimensions (104,)


In [12]:
# Lets train a Decision Tree Classifier on Petal Length and Width
feat=[0,1]
dt=DecisionTree(0.95,5)
dt.train(Xtrain,Ytrain)

In [13]:
#Lets test it on the set of unseen examples...
pclasses=dt.predict(Xtest)

# Let's See How Good we are doing

In [14]:
#Lets see how good we are doing, by finding the accuracy on the test set..
print (np.sum(pclasses==Ytest))
print ("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

41
Accuracy =  0.9111111111111111


# Lets Train on All 4 Features and all 3 classes 

In [15]:
# Split your data into training and test-set... 
# see the documentation of split_data in tools for further information...
Xtrain,Ytrain,Xtest,Ytest=t.split_data(X,Y)

print (" Training Data Set Dimensions=", Xtrain.shape, "Training True Class labels dimensions", Ytrain.shape)
print (" Test Data Set Dimensions=", Xtest.shape, "Test True Class labels dimensions", Ytrain.shape)   

 Training Data Set Dimensions= (104, 4) Training True Class labels dimensions (104,)
 Test Data Set Dimensions= (45, 4) Test True Class labels dimensions (104,)


In [16]:
dt=DecisionTree(0.95,5)
dt.train(Xtrain,Ytrain)
pclasses=dt.predict(Xtest)
#Lets see how good we are doing, by finding the accuracy on the test set..
print (np.sum(pclasses==Ytest))
print ("Accuracy = ", np.sum(pclasses==Ytest)/float(Ytest.shape[0]))

42
Accuracy =  0.9333333333333333
